In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unrar x '/content/drive/MyDrive/Cheating Detection/PersianPlagdet2016-text-alignment-corpus.rar'

In [1]:
import os

folder_path = ['PersianPlagdet2016-text-alignment-train-corpus/src/', 'PersianPlagdet2016-text-alignment-train-corpus/susp/']

txt_files = []

for folder in folder_path:
  for root, dirs, files in os.walk(folder):
      for file in files:
          if file.endswith(".txt"):
              txt_files.append(os.path.join(root, file))

In [2]:
len(txt_files)

3088

In [3]:
import io


txt_collection = {}

for file in txt_files:
    with io.open(file, 'r', encoding='utf-8') as f:
        txt_collection[file] = f.read()

In [ ]:
!pip install hazm

In [4]:
from hazm import *
import tqdm



hazm_normalizer = Normalizer()

txt_sentences = {}
txt_norm_tokenize = {}

for file in tqdm.tqdm(txt_collection):

    normalized_txt = hazm_normalizer.normalize(txt_collection[file])
    sentences = sent_tokenize(normalized_txt)
    txt_sentences[file] = sentences
    temp = []

    for sent in sentences:
        temp += [word_tokenize(sent)]

    txt_norm_tokenize[file] = temp

100%|██████████| 3088/3088 [00:46<00:00, 66.33it/s] 


In [6]:
import codecs

stop_path = '/content/drive/MyDrive/Cheating Detection/persian_stopwords.txt'
stopwords = [hazm_normalizer.normalize(x.strip()) for x in codecs.open(stop_path,'r','utf-8').readlines()]

In [7]:
lemmatizer = Lemmatizer()


all_tokens_nonstop = {}

for file in tqdm.tqdm(txt_norm_tokenize):
    temp = []
    for token_list in txt_norm_tokenize[file]:
        temp2 = []
        temp2 += [lemmatizer.lemmatize(t.replace("_", "")) for t in token_list if t not in stopwords]
        temp += [temp2]
    all_tokens_nonstop[file] = temp

100%|██████████| 3088/3088 [00:22<00:00, 138.70it/s]


In [9]:
file = open('/content/PersianPlagdet2016-text-alignment-train-corpus/pairs', "r")

content = file.read()

file.close()

In [10]:
pairs_dict = {'suspicious': [], 'source': []}

for row in content.split('\n'):
    temp = row.split(' ')
    if len(temp) == 2:
        susp, src = temp
    pairs_dict['suspicious'] += [susp]
    pairs_dict['source'] += [src]

In [11]:
import pandas as pd


df = pd.DataFrame(pairs_dict)

In [12]:
df

,suspicious,source
0,suspicious-document00001100017.txt,source-document00001222909.txt
1,suspicious-document00001100593.txt,source-document00001183668.txt
2,suspicious-document0000110086.txt,source-document00001405402.txt
3,suspicious-document00001101968.txt,source-document00001213543.txt
4,suspicious-document00001102809.txt,source-document00001320655.txt
...,...,...
2745,suspicious-document0000187571.txt,source-document000012725.txt
2746,suspicious-document0000194397.txt,source-document00001307951.txt
2747,suspicious-document0000194397.txt,source-document00001308967.txt
2748,suspicious-document0000198379.txt,source-document00001293130.txt


In [131]:
Documents_paths = []

susp_path = os.path.join(folder_path[1], df.iloc[3, 0])
plag_path = os.path.join(folder_path[0], df.iloc[3, 1])

Documents_paths += [susp_path]

for i in df.iloc[:, 1]:
    Documents_paths += [os.path.join(folder_path[0], i)]

Documents_paths = list(dict.fromkeys(Documents_paths))

In [130]:
susp_path

'PersianPlagdet2016-text-alignment-train-corpus/susp/suspicious-document00001101968.txt'

In [132]:
plag_path

'PersianPlagdet2016-text-alignment-train-corpus/src/source-document00001213543.txt'

In [133]:
Documents = []
sent_len = {}

temp = 0
for path in Documents_paths:
    sent_len[path] = [temp, len(all_tokens_nonstop[path]) + temp]
    temp += len(all_tokens_nonstop[path])
    Documents += all_tokens_nonstop[path]

In [134]:
from sklearn.feature_extraction.text import TfidfVectorizer


sentences_str = [' '.join(sentence) for sentence in Documents]
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(sentences_str)
feature_names = vectorizer.get_feature_names_out()

In [135]:
feature_names

array(['aa', 'aassrec', 'ab', ..., 'ﯾازده', 'ﯾک', 'ﯾکﯽ'], dtype=object)

In [136]:
len(feature_names)

40543

In [137]:
tfidf_matrix[0].toarray()[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [138]:
tfidf_matrix

<42284x40543 sparse matrix of type '<class 'numpy.float64'>'
	with 617185 stored elements in Compressed Sparse Row format>

In [139]:
sent_len[susp_path]

[0, 15]

In [140]:
susp_doc = tfidf_matrix[:sent_len[susp_path][1], :]
src_docs = tfidf_matrix[sent_len[susp_path][1]:, :]

In [141]:
from sklearn.metrics.pairwise import cosine_similarity


cos_sim = cosine_similarity(susp_doc, src_docs)

In [142]:
cos_sim.shape

(15, 42269)

In [143]:
import numpy as np

threshold = 0.25
indices = np.argwhere(cos_sim > threshold)

str_idx = []
for index in indices:
    i, j = index
    str_idx += [j]
    print(f"Cosine similarity between row {i} of matrix A and row {j} of matrix B: {cos_sim[i, j]}")

Cosine similarity between row 0 of matrix A and row 19618 of matrix B: 0.3094362425686463
Cosine similarity between row 1 of matrix A and row 9273 of matrix B: 0.4103893048395495
Cosine similarity between row 1 of matrix A and row 9291 of matrix B: 0.32912273976486706
Cosine similarity between row 1 of matrix A and row 17461 of matrix B: 0.3346487096153997
Cosine similarity between row 1 of matrix A and row 20315 of matrix B: 0.2694985907722104
Cosine similarity between row 2 of matrix A and row 517 of matrix B: 0.28791150048817926
Cosine similarity between row 2 of matrix A and row 9291 of matrix B: 0.27312470661234955
Cosine similarity between row 2 of matrix A and row 11710 of matrix B: 0.39109441074929874
Cosine similarity between row 2 of matrix A and row 13886 of matrix B: 0.3053352683421994
Cosine similarity between row 2 of matrix A and row 24933 of matrix B: 0.28265503611366233
Cosine similarity between row 2 of matrix A and row 30888 of matrix B: 0.25756259242273627
Cosine si

In [145]:
cheat_src = {}

for idx in sorted(str_idx):
    for path, rng in sent_len.items():
        if rng[0] <= idx <= rng[1]:
            if path not in cheat_src:
                cheat_src[path] = 1
            else:
                cheat_src[path] += 1

In [147]:
cheat_src

{'PersianPlagdet2016-text-alignment-train-corpus/src/source-document00001213543.txt': 1,
 'PersianPlagdet2016-text-alignment-train-corpus/src/source-document00001189284.txt': 1,
 'PersianPlagdet2016-text-alignment-train-corpus/src/source-document00001310293.txt': 1,
 'PersianPlagdet2016-text-alignment-train-corpus/src/source-document00001114500.txt': 1,
 'PersianPlagdet2016-text-alignment-train-corpus/src/source-document00001383258.txt': 1,
 'PersianPlagdet2016-text-alignment-train-corpus/src/source-document00001192284.txt': 1,
 'PersianPlagdet2016-text-alignment-train-corpus/src/source-document00001463828.txt': 2,
 'PersianPlagdet2016-text-alignment-train-corpus/src/source-document00001421423.txt': 1,
 'PersianPlagdet2016-text-alignment-train-corpus/src/source-document00001144829.txt': 1,
 'PersianPlagdet2016-text-alignment-train-corpus/src/source-document00001108008.txt': 2,
 'PersianPlagdet2016-text-alignment-train-corpus/src/source-document00001118630.txt': 1,
 'PersianPlagdet2016-

In [148]:
cheat_src[plag_path]

1

In [149]:
df[df['source'] == plag_path.split('/')[-1]]

,suspicious,source
3,suspicious-document00001101968.txt,source-document00001213543.txt
709,suspicious-document00001528053.txt,source-document00001213543.txt
1628,suspicious-document00001200433.txt,source-document00001213543.txt


In [151]:
df[df['suspicious'] == susp_path.split('/')[-1]]

,suspicious,source
3,suspicious-document00001101968.txt,source-document00001213543.txt
